In [1]:
!pip install easyocr


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install db-sqlite3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import easyocr
import re
import tempfile
import sqlite3
import pandas as pd
from PIL import Image


In [4]:
from google.colab import files
img = files.upload()

Saving 1.png to 1.png


In [5]:
with tempfile.NamedTemporaryFile(delete=False) as temp_file:
    temp_file.write(img[next(iter(img))])
    temp_path = temp_file.name
   

In [6]:
reader = easyocr.Reader(['en'])


In [7]:
result = reader.readtext(temp_path,detail=0)
print(result)

['Selva', 'DATA MANAGER', '+123-456-7890', '+123-456-7891', 'WWW XYZI.com', 'hello@XYZ1.com', '123 ABC St , Chennai;', 'selva', 'TamilNadu 600113', 'digitals']


In [8]:
list = result
print(list)

['Selva', 'DATA MANAGER', '+123-456-7890', '+123-456-7891', 'WWW XYZI.com', 'hello@XYZ1.com', '123 ABC St , Chennai;', 'selva', 'TamilNadu 600113', 'digitals']


In [9]:
num = '(\+||s)(\d+\-)'
mail = '\w+@\w+\.com'
website = '\w+(.com|com)'
add = '(\d+\w)'
pin = '(\w+)\s(\d{6})'

In [10]:
f = []
for i in list:
  if re.match(num,i) or re.findall(mail,i) or re.findall(website,i) or re.match(add,i) or re.match(pin,i) :
    f.append(i)

In [11]:
number = []
web = []
mailid = []
address = []
state = []
for i in list:
  if re.findall(num,i):
    number.append(i)
  elif re.findall(mail,i):
    mailid.append(i)
  elif re.findall(website,i):
    web.append(i)
  elif re.match(add,i):
    address.append(i)
  elif re.match(pin,i):
    state.append(i)

In [12]:
print(state)

['TamilNadu 600113']


In [13]:
if state == []:
  address = ''.join(address)
  parts = address.split(';')
  address = parts[0]
  state = parts[1]

In [14]:
number = ''.join(number)
web = ''.join(web)
mailid = ''.join(mailid)
address = ''.join(address)
state = ''.join(state)

In [15]:
print(f)
print(list)

['+123-456-7890', '+123-456-7891', 'WWW XYZI.com', 'hello@XYZ1.com', '123 ABC St , Chennai;', 'TamilNadu 600113']
['Selva', 'DATA MANAGER', '+123-456-7890', '+123-456-7891', 'WWW XYZI.com', 'hello@XYZ1.com', '123 ABC St , Chennai;', 'selva', 'TamilNadu 600113', 'digitals']


In [16]:
f2 =[]
for i in list:
  if i not in f:
    f2.append(i)
print(f2)

['Selva', 'DATA MANAGER', 'selva', 'digitals']


In [17]:
name = f2[0]
role = f2[1]
if len(f2) == 4 :
  company = f2[2] + ' ' + f2[3]
else:
  company = f2[2]



In [18]:
data = (name,role,company,number,mailid,web,address,state)
print(data)


('Selva', 'DATA MANAGER', 'selva digitals', '+123-456-7890+123-456-7891', 'hello@XYZ1.com', 'WWW XYZI.com', '123 ABC St , Chennai;', 'TamilNadu 600113')


In [19]:


# Connect to the database (create a new one if it doesn't exist)
conn = sqlite3.connect('cards.db')

# Create a new table to store the data
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS mytable
             (name varchar(50), role varchar(50), company varchar(50), number varchar(50),
              mailid varchar(50), web varchar(50), address varchar(50), state varchar(50))''')

# Insert the data into the table

c.execute("INSERT INTO mytable VALUES (?, ?, ?, ?, ?, ?, ?, ?)", data)

# Commit the changes 
conn.commit()



In [20]:
df = pd.read_sql_query('SELECT * FROM mytable', conn)

In [21]:
print(df)

    name          role         company                      number  \
0  Selva  DATA MANAGER  selva digitals  +123-456-7890+123-456-7891   

           mailid           web                address             state  
0  hello@XYZ1.com  WWW XYZI.com  123 ABC St , Chennai;  TamilNadu 600113  
